In [1]:
import pandas as pd
import numpy as np


In [2]:
fear=pd.read_csv("/content/fear_greed_index.csv")
historical=pd.read_csv("/content/historical_data.csv")

/tmp/ipython-input-1019618819.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  historical=pd.read_csv("/content/historical_data.csv")


In [3]:
fear.head()

,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


In [4]:
historical.head()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,5.201771e+10,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,5.201771e+10,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,5.201771e+10,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,5.201771e+10,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,5.201771e+10,True,0.003055,1.050000e+15,1.730000e+12


In [10]:
print(fear.duplicated().sum())
print(fear.isnull().sum())
print(fear["classification"].unique())

0
timestamp         0
value             0
classification    0
date              0
dtype: int64
['Fear' 'Extreme Fear' 'Neutral' 'Greed' 'Extreme Greed']


In [9]:
print(historical.duplicated().sum())
print(historical.isnull().sum())

0
Account             0
Coin                0
Execution Price     0
Size Tokens         0
Size USD            0
Side                0
Timestamp IST       0
Start Position      1
Direction           1
Closed PnL          1
Transaction Hash    1
Order ID            1
Crossed             1
Fee                 1
Trade ID            1
Timestamp           1
dtype: int64


In [11]:
fear["date"] = pd.to_datetime(fear["date"])
sentiment_map = {"Neutral":0,"Extreme Fear": 1,"Fear": 2,"Greed": 3,"Extreme Greed": 4}
fear["sentiment_score"] = fear["classification"].map(sentiment_map)

In [14]:
historical["Timestamp IST"]=historical["Timestamp IST"].astype(str)
historical["Timestamp IST"]=pd.to_datetime(historical["Timestamp IST"],errors="coerce",dayfirst=True)
historical=historical.loc[historical["Timestamp IST"].notna()].copy()
historical["trade_date"]=pd.to_datetime(historical["Timestamp IST"].dt.date)
historical["Start Position"]=historical["Start Position"].fillna(0)
historical["Direction"]=historical["Direction"].fillna(historical["Side"])
historical["Closed PnL"]=historical["Closed PnL"].fillna(0)
historical["Fee"]=historical["Fee"].fillna(historical["Fee"].median())

In [16]:
newdata=historical.merge(fear[["date", "classification", "sentiment_score"]],left_on="trade_date",right_on="date",how="left")
newdata["net_pnl"]=newdata["Closed PnL"]-newdata["Fee"]
newdata["is_profitable"]=(newdata["net_pnl"]>0).astype(int)
newdata["risk_efficiency"]=newdata["net_pnl"]/newdata["Size USD"]


In [17]:
sentiment_summary=newdata.groupby("classification").agg(avg_net_pnl=("net_pnl", "mean"),win_rate=("is_profitable", "mean"),
    avg_trade_size=("Size USD", "mean"),avg_fee=("Fee", "mean"),avg_risk_efficiency=("risk_efficiency", "mean"),trade_count=("net_pnl", "count"))
print(sentiment_summary)


                avg_net_pnl  win_rate  avg_trade_size   avg_fee  \
classification                                                    
Extreme Fear      31.865232  0.351628     6196.106960  1.272707   
Extreme Greed     68.543742  0.467568     3168.979574  0.725812   
Fear              61.801340  0.426629     9517.225518  1.767202   
Greed             33.685191  0.386083     6153.372590  1.369820   
Neutral           44.887804  0.415900     5535.893674  1.182677   

                avg_risk_efficiency  trade_count  
classification                                    
Extreme Fear               0.003225        13574  
Extreme Greed              0.043647        33640  
Fear                       0.017379        43989  
Greed                      0.019945        39678  
Neutral                    0.012398        27735  


In [18]:
from scipy.stats import mannwhitneyu, chi2_contingency
fear_trades=newdata[newdata["sentiment_score"]<=1]["risk_efficiency"].dropna()
greed_trades=newdata[newdata["sentiment_score"]>=2]["risk_efficiency"].dropna()
value,p_value=mannwhitneyu(fear_trades,greed_trades,alternative="two-sided")
contingency=pd.crosstab(newdata["classification"],newdata["is_profitable"])
a,chi_p,b,c=chi2_contingency(contingency)

print("P_value:",p_value)
print("Chi_p:",chi_p)

P_value: 0.00010228846101905791
Chi_p: 2.3640344535796304e-164


In [19]:
buy_sell_summary=newdata.groupby(["classification", "Side"]).agg(avg_net_pnl=("net_pnl", "mean"),
    win_rate=("is_profitable", "mean"),trade_count=("net_pnl", "count"))
print(buy_sell_summary)

                     avg_net_pnl  win_rate  trade_count
classification Side                                    
Extreme Fear   BUY     36.324309  0.186880         7454
               SELL    26.434193  0.552288         6120
Extreme Greed  BUY     10.646444  0.357407        15148
               SELL   115.971185  0.557809        18492
Fear           BUY     77.585054  0.236591        21552
               SELL    46.640196  0.609172        22437
Greed          BUY      7.556894  0.319307        19915
               SELL    60.014444  0.453372        19763
Neutral        BUY     45.317362  0.243022        14402
               SELL    44.423806  0.602640        13333
